# Chapter 06-03: RNN, LSTM, GRU

## 학습 목표
- 순환 신경망(RNN)의 작동 원리와 기울기 소실 문제를 이해한다.
- LSTM의 게이트 메커니즘을 수식으로 파악한다.
- GRU의 간소화된 구조를 이해한다.
- `return_sequences`, Bidirectional, Stacked 구조를 실습한다.
- RNN/LSTM/GRU의 파라미터 수와 성능을 비교한다.

## 목차
1. [기본 임포트](#1.-기본-임포트)
2. [수식 정리](#2.-수식-정리)
3. [기울기 소실 문제](#3.-기울기-소실-문제)
4. [SimpleRNN vs LSTM vs GRU 비교](#4.-비교)
5. [return_sequences](#5.-return_sequences)
6. [Bidirectional LSTM](#6.-Bidirectional-LSTM)
7. [Stacked LSTM](#7.-Stacked-LSTM)
8. [정리](#8.-정리)

---
### 🐣 초등학생을 위한 RNN/LSTM/GRU 친절 설명!

#### 🔄 RNN - 기억을 가진 신경망!

> 💡 **비유**: 소설을 읽을 때 앞 내용을 기억하며 읽는 것처럼!
> 일반 신경망은 각 입력을 독립적으로 처리해요.
> RNN은 이전 출력을 다음 입력에 **함께** 사용해요.

```
입력:  "나는" → "오늘" → "학교에" → "갔다"
         ↓       ↓        ↓         ↓
RNN:  [h₁]──→[h₂]──→[h₃]──→[h₄]  ← 이전 상태(기억)가 전달됨!
```

#### 😵 RNN의 문제: 장기 의존성(Long-term Dependency)

긴 문장에서 앞부분을 기억하기 어려워요!
기울기가 역전파 중 점점 작아져 **기울기 소실(Vanishing Gradient)** 발생.

#### 🏆 LSTM - 선택적 기억 장치!

LSTM은 **게이트(Gate)**로 무엇을 기억하고 잊을지 선택해요:

| 게이트 | 역할 | 비유 |
|--------|------|------|
| **Forget Gate** | 기존 기억 중 버릴 것 결정 | 불필요한 정보 지우기 🗑️ |
| **Input Gate** | 새 정보 중 저장할 것 결정 | 중요한 것만 노트에 기록 📝 |
| **Output Gate** | 현재 기억에서 출력할 것 결정 | 지금 필요한 것만 꺼내기 📤 |

#### ⚡ GRU - LSTM의 경량 버전!

게이트를 2개로 단순화 → 더 빠르고 파라미터 적음!
LSTM만큼 성능이 좋은 경우도 많아서 실무에서 자주 사용해요.

> 💡 **언제 뭘 써요?**
> - 빠른 실험: **GRU**
> - 정확도 중요: **LSTM**
> - 아주 긴 시퀀스: **Transformer (Attention 기반)**


In [ ]:
# 기본 라이브러리 임포트
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# 한글 폰트 설정 (macOS)
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

print(f"TensorFlow 버전: {tf.__version__}")

## 2. 수식 정리

### 2.1 SimpleRNN

시간 스텝 $t$에서의 은닉 상태 $h_t$:

$$h_t = \tanh(W_h h_{t-1} + W_x x_t + b)$$

- $x_t$: 현재 시간 스텝의 입력
- $h_{t-1}$: 이전 시간 스텝의 은닉 상태
- $W_h$, $W_x$: 학습 가중치 행렬
- $b$: 편향(bias)

---

### 2.2 LSTM (Long Short-Term Memory)

LSTM은 **망각 게이트($f_t$)**, **입력 게이트($i_t$)**, **출력 게이트($o_t$)**와  
**셀 상태($C_t$)**를 추가하여 장기 의존성을 학습한다.

**망각 게이트** - 이전 셀 상태에서 무엇을 잊을지 결정:
$$f_t = \sigma(W_f[h_{t-1}, x_t] + b_f)$$

**입력 게이트** - 새 정보에서 무엇을 저장할지 결정:
$$i_t = \sigma(W_i[h_{t-1}, x_t] + b_i)$$

**셀 상태 업데이트** - 이전 셀 상태를 망각/갱신:
$$C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t$$

**출력 게이트** - 은닉 상태로 무엇을 출력할지 결정:
$$o_t = \sigma(W_o[h_{t-1}, x_t] + b_o)$$

**은닉 상태 업데이트**:
$$h_t = o_t \odot \tanh(C_t)$$

여기서 $\odot$는 요소별 곱(Hadamard product), $\sigma$는 시그모이드 함수이다.

---

### 2.3 GRU (Gated Recurrent Unit)

GRU는 LSTM을 간소화하여 **업데이트 게이트($z_t$)**와 **리셋 게이트($r_t$)**만 사용한다.

**업데이트 게이트** - 이전 상태와 새 상태를 얼마나 섞을지:
$$z_t = \sigma(W_z[h_{t-1}, x_t])$$

**은닉 상태 업데이트**:
$$h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t$$

$z_t = 0$이면 이전 상태를 그대로 유지, $z_t = 1$이면 새 상태로 완전 교체.

## 3. 기울기 소실 문제와 장기 의존성

### SimpleRNN의 한계

역전파(BPTT: Backpropagation Through Time) 과정에서  
긴 시퀀스를 다룰 때 기울기가 지수적으로 작아지는 **기울기 소실(Vanishing Gradient)** 문제가 발생한다.

$$\frac{\partial L}{\partial h_0} = \prod_{t=1}^{T} \frac{\partial h_t}{\partial h_{t-1}}$$

각 항이 1보다 작으면 $T$가 클수록 기울기가 $\approx 0$이 된다.

### 장기 의존성(Long-Term Dependency)

예시 문장:
> "I grew up in France... (중간에 50개 단어) ...so I speak fluent **French**."

"French"를 예측하려면 멀리 앞의 "France"를 기억해야 하지만,  
SimpleRNN은 이 정보를 유지하지 못한다.

### LSTM과 GRU의 해결책
- **LSTM**: 셀 상태($C_t$)라는 별도의 경로로 기울기가 흐를 수 있어 소실 완화
- **GRU**: LSTM보다 단순하지만 유사한 효과, 파라미터 수 감소

In [ ]:
# SimpleRNN, LSTM, GRU 레이어 비교

# 공통 설정
VOCAB_SIZE   = 10000
EMBED_DIM    = 64
HIDDEN_UNITS = 64
SEQ_LEN      = 100

def build_model(rnn_type, name):
    """RNN 유형에 따른 모델 생성"""
    inputs = tf.keras.Input(shape=(SEQ_LEN,), name='input')
    x = tf.keras.layers.Embedding(VOCAB_SIZE, EMBED_DIM)(inputs)
    
    if rnn_type == 'SimpleRNN':
        # SimpleRNN: 가장 단순한 순환 구조
        x = tf.keras.layers.SimpleRNN(HIDDEN_UNITS)(x)
    elif rnn_type == 'LSTM':
        # LSTM: 3개 게이트, 셀 상태 추가
        x = tf.keras.layers.LSTM(HIDDEN_UNITS)(x)
    elif rnn_type == 'GRU':
        # GRU: 2개 게이트, LSTM보다 경량
        x = tf.keras.layers.GRU(HIDDEN_UNITS)(x)
    
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    return tf.keras.Model(inputs, outputs, name=name)

# 세 가지 모델 생성
models = {
    'SimpleRNN': build_model('SimpleRNN', 'SimpleRNN_Model'),
    'LSTM':      build_model('LSTM',      'LSTM_Model'),
    'GRU':       build_model('GRU',       'GRU_Model'),
}

print("=" * 55)
print(f"{'모델':12s} {'총 파라미터':>15s} {'RNN 레이어 파라미터':>20s}")
print("=" * 55)

for name, model in models.items():
    total_params = model.count_params()
    # RNN 레이어만의 파라미터 계산
    rnn_layer = [l for l in model.layers if name in type(l).__name__][0]
    rnn_params = rnn_layer.count_params()
    print(f"{name:12s} {total_params:>15,d} {rnn_params:>20,d}")

print("=" * 55)
print()
print("파라미터 수 비교 (은닉 유닛 h=64, 입력 x=64):")
h, x = HIDDEN_UNITS, EMBED_DIM
print(f"  SimpleRNN: (h+x+1)×h = ({h}+{x}+1)×{h} = {(h+x+1)*h:,d}")
print(f"  LSTM:      4×(h+x+1)×h = 4×{(h+x+1)*h:,d} = {4*(h+x+1)*h:,d}  (게이트 4개)")
print(f"  GRU:       3×(h+x+1)×h = 3×{(h+x+1)*h:,d} = {3*(h+x+1)*h:,d}  (게이트 3개)")

## 5. return_sequences=True vs False

RNN 계열 레이어에서 `return_sequences` 파라미터는 출력 형태를 결정한다.

| `return_sequences` | 출력 형태 | 용도 |
|--------------------|-----------|------|
| `False` (기본값) | `(batch, hidden_units)` | 마지막 타임스텝만 출력, 분류 등 |
| `True` | `(batch, timesteps, hidden_units)` | 모든 타임스텝 출력, 스택 RNN, Seq2Seq 등 |

In [ ]:
# return_sequences 차이 실습

BATCH_SIZE = 2
SEQ_LEN    = 5
INPUT_DIM  = 8   # 각 타임스텝의 입력 차원
UNITS      = 4   # LSTM 은닉 유닛 수

# 샘플 입력 데이터: (배치, 시퀀스 길이, 입력 차원)
sample_input = tf.random.normal((BATCH_SIZE, SEQ_LEN, INPUT_DIM))
print(f"입력 형태: {sample_input.shape} → (배치, 시퀀스 길이, 입력 차원)")
print()

# return_sequences=False (기본값): 마지막 타임스텝만 반환
lstm_false = tf.keras.layers.LSTM(UNITS, return_sequences=False)
output_false = lstm_false(sample_input)
print(f"return_sequences=False: {output_false.shape}")
print(f"  → (배치={BATCH_SIZE}, 은닉={UNITS}) - 마지막 타임스텝만")
print()

# return_sequences=True: 모든 타임스텝 반환
lstm_true = tf.keras.layers.LSTM(UNITS, return_sequences=True)
output_true = lstm_true(sample_input)
print(f"return_sequences=True: {output_true.shape}")
print(f"  → (배치={BATCH_SIZE}, 시퀀스={SEQ_LEN}, 은닉={UNITS}) - 모든 타임스텝")
print()

# 시각화: 각 타임스텝의 LSTM 출력값
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# return_sequences=False: 마지막 출력만
axes[0].bar(range(UNITS), output_false[0].numpy())
axes[0].set_title(f"return_sequences=False\n출력 형태: {output_false.shape}")
axes[0].set_xlabel("은닉 유닛 인덱스")
axes[0].set_ylabel("활성화 값")

# return_sequences=True: 모든 타임스텝 출력
im = axes[1].imshow(output_true[0].numpy(), cmap='RdBu', aspect='auto', vmin=-1, vmax=1)
axes[1].set_title(f"return_sequences=True\n출력 형태: {output_true.shape}")
axes[1].set_xlabel("은닉 유닛 인덱스")
axes[1].set_ylabel("타임스텝")
axes[1].set_yticks(range(SEQ_LEN))
axes[1].set_yticklabels([f"t={i}" for i in range(SEQ_LEN)])
plt.colorbar(im, ax=axes[1])

plt.suptitle("LSTM return_sequences 비교", fontsize=13)
plt.tight_layout()
plt.show()

## 6. Bidirectional LSTM

단방향 LSTM은 왼쪽→오른쪽(순방향) 문맥만 학습한다.  
**양방향 LSTM(Bidirectional LSTM)**은 순방향과 역방향 두 LSTM을 병렬로 실행한 뒤  
출력을 연결(concat)하여 **양쪽 문맥**을 모두 반영한다.

$$\vec{h_t} = \text{LSTM}_{\text{forward}}(x_1, \ldots, x_t)$$
$$\overleftarrow{h_t} = \text{LSTM}_{\text{backward}}(x_T, \ldots, x_t)$$
$$h_t = [\vec{h_t}; \overleftarrow{h_t}]$$

출력 차원은 단방향의 2배가 된다.

In [ ]:
# Bidirectional LSTM 실습

# 샘플 입력
sample_input = tf.random.normal((2, 10, 16))  # (배치, 시퀀스, 특성)

# 단방향 LSTM
unidirectional = tf.keras.layers.LSTM(32, return_sequences=True)
uni_output = unidirectional(sample_input)

# Bidirectional LSTM
# merge_mode: 'concat'(기본), 'sum', 'mul', 'ave'
bidirectional = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(32, return_sequences=True),
    merge_mode='concat'  # 순방향+역방향 결합 방식
)
bi_output = bidirectional(sample_input)

print(f"입력 형태:              {sample_input.shape}")
print(f"단방향 LSTM 출력:      {uni_output.shape}  → (배치, 시퀀스, 32)")
print(f"양방향 LSTM 출력:      {bi_output.shape}  → (배치, 시퀀스, 32×2=64)")
print()

# Bidirectional LSTM을 사용한 텍스트 분류 모델
bi_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 64, mask_zero=True),
    # Bidirectional로 감싸면 양방향 학습
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
], name='BiLSTM_Classifier')

bi_model.build(input_shape=(None, 100))
bi_model.summary()

## 7. Stacked LSTM (다층 LSTM)

In [ ]:
# Stacked LSTM: 여러 층의 LSTM을 쌓는 방법
# 첫 번째 LSTM은 return_sequences=True로 모든 타임스텝을 다음 레이어에 전달

stacked_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 64, mask_zero=True),
    
    # 첫 번째 LSTM: return_sequences=True → 다음 LSTM에 전체 시퀀스 전달
    tf.keras.layers.LSTM(128, return_sequences=True, name='lstm_1'),
    tf.keras.layers.Dropout(0.3),
    
    # 두 번째 LSTM: return_sequences=False → 마지막 타임스텝만 출력
    tf.keras.layers.LSTM(64, return_sequences=False, name='lstm_2'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
], name='Stacked_LSTM')

stacked_model.build(input_shape=(None, 100))
stacked_model.summary()
print()
print("[핵심] 첫 번째 LSTM에서 return_sequences=True를 설정해야")
print("       두 번째 LSTM이 (배치, 시퀀스, 특성) 형태의 입력을 받을 수 있다.")

## 8. 정리 - RNN/LSTM/GRU 비교 표

| 항목 | SimpleRNN | LSTM | GRU |
|------|-----------|------|-----|
| **게이트 수** | 없음 | 3개 (f, i, o) | 2개 (z, r) |
| **셀 상태** | 없음 | 있음 ($C_t$) | 없음 |
| **파라미터 수** | 적음 | 많음 (4배) | 중간 (3배) |
| **장기 의존성** | 약함 | 강함 | 강함 |
| **학습 속도** | 빠름 | 느림 | 중간 |
| **권장 사용처** | 짧은 시퀀스 | 긴 시퀀스 | 긴 시퀀스 (경량) |

### 선택 가이드
- 짧은 시퀀스이고 속도가 중요: **SimpleRNN**
- 긴 시퀀스이고 정확도가 중요: **LSTM**
- 긴 시퀀스이고 속도도 중요: **GRU**
- 문맥이 양방향으로 필요: **Bidirectional LSTM/GRU**

### 다음 챕터 예고
- **Chapter 06-04**: 텍스트 분류 (Text Classification)  
  IMDB 데이터셋으로 감성 분석 모델을 구현하고 비교한다.